In [1]:
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader 
from skimage.metrics import structural_similarity as ssim
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from matplotlib.lines import Line2D

from AE_functions import *
from eval_functions import *
from make_dataset import * 

%matplotlib qt

import warnings
warnings.filterwarnings("ignore")


In [2]:
## Define which training model, that should be evaluated

train = '52530'

_____________________________

In [50]:
## Loading test image + outlier image

# Loading data (testset, 41 elements)
img_dir_test = "G:/Mit drev/Uni/6. semester/JK_bachelor/Data/Verse20/Outlier_detection/crops_validation_prep/img"
heatmap_dir_test = "G:/Mit drev/Uni/6. semester/JK_bachelor/Data/Verse20/Outlier_detection/crops_validation_prep/heatmaps"
msk_dir_test = "G:/Mit drev/Uni/6. semester/JK_bachelor/Data/Verse20/Outlier_detection/crops_validation_prep/msk"

VerSe_test = LoadData(img_dir=img_dir_test, msk_dir = msk_dir_test, distfield_dir=heatmap_dir_test)
test_loader = DataLoader(VerSe_test, batch_size=1, shuffle=False, num_workers=0)

healthy, outlier = healthy_outlier(test_loader)

# Create datasets
dataset_healthy = generate_dataset(healthy, 120)
dataset_outlier = generate_dataset_outlier(outlier, 120, radius=30, mode="")


In [51]:
## Evaluating the model with new data (healthy and outliers)

# Loading the model
model_path = f'G:/Mit drev/Uni/6. semester/JK_bachelor/{train}_training/model_conv_999.pth'

model = load_model(model_path, [1, 16, 32, 64, 128])

# Testing the model with MSE
error_healthy = []
error_outlier = []

for i in range(len(dataset_healthy)):
    # test_healthy, _, _  = test_loader.dataset[i]
    test_healthy = dataset_healthy[i]
    loss = evaluate_model(model, test_healthy[0].unsqueeze(dim=0))
    error_healthy.append(loss)

    # test_outlier, _, _ = test_loader_o.dataset[i]
    test_outlier = dataset_outlier[i]
    loss = evaluate_model(model, test_outlier[0].unsqueeze(dim=0))
    error_outlier.append(loss)

-----

# MSE

In [52]:
## Make histogram


hist_values1, hist_values2, bins1, bins2 = plot_histograms(error_healthy, error_outlier, 10)

mean_healthy = np.mean(error_healthy)
std_healthy = np.std(error_healthy)

mean_outlier = np.mean(error_outlier)
std_outlier = np.std(error_outlier)

In [53]:
print(f"Mean healthy: {mean_healthy}, std healthy: {std_healthy}")
print(f"Mean outlier: {mean_outlier}, std outlier: {std_outlier}")

Mean healthy: 9.381925178786332e-05, std healthy: 5.115464393391812e-05
Mean outlier: 0.00026636500473008103, std outlier: 0.0001339304350886752


In [54]:
## Plotting the difference from outlier input to reconstructions
org_img = dataset_outlier[0].squeeze()
reconstructed_img = model(dataset_outlier[0].unsqueeze(dim=0)).squeeze().detach().numpy()

fig, ax = plt.subplots(1, 3, figsize=(12, 4))

ax[0].imshow(dataset_outlier[0].squeeze(), cmap='gray')
ax[0].set_title('Original Image')

ax[1].imshow(reconstructed_img, cmap='gray')
ax[1].set_title('Reconstructed Image')

ax[2].imshow(org_img - reconstructed_img, cmap='bwr')
ax[2].set_title('Difference')
fig.colorbar(ax[2].imshow(org_img - reconstructed_img, cmap='bwr'))

plt.show()


------

# SSIM

In [55]:
## Evaluating the model with new data (healthy and outliers)

# Testing the model
ssim_healthy = []
ssim_outlier = []

for i in range(len(dataset_healthy)):
    # test_healthy, _, _  = test_loader.dataset[i]
    test_healthy = dataset_healthy[i]
    loss = evaluate_model_SSIM(model, test_healthy[0].unsqueeze(dim=0))
    ssim_healthy.append(loss)

    # test_outlier, _, _ = test_loader_o.dataset[i]
    test_outlier = dataset_outlier[i]
    loss = evaluate_model_SSIM(model, test_outlier[0].unsqueeze(dim=0))
    ssim_outlier.append(loss)

hist_values1, hist_values2, bins1, bins2 = plot_histograms_SSIM(ssim_healthy, ssim_outlier, 15)

In [56]:
## Plotting the difference from outlier input to reconstructions

org_img = dataset_outlier[0].squeeze()
org_img_np = org_img.cpu().detach().numpy()

reconstructed_img = model(dataset_outlier[0].unsqueeze(dim=0)).squeeze().detach().numpy()
reconstructed_img_np = reconstructed_img.astype(np.float64)


# Compute SSIM
mssim, ssim_map  = ssim(org_img_np, reconstructed_img_np, full=True) # full=True returns the full structural similarity image
print('The mean structural similarity index over the image is:',mssim)
#plot
fig, ax = plt.subplots(1, 3, figsize=(12, 4))

ax[0].imshow(org_img_np, cmap='gray')
ax[0].set_title('Original Image')

ax[1].imshow(reconstructed_img_np, cmap='gray')
ax[1].set_title('Reconstructed Image')

ax[2].imshow(ssim_map, cmap='bwr')
ax[2].set_title('SSIM Map')
fig.colorbar(ax[2].imshow(ssim_map, cmap='bwr'))

plt.show()

The mean structural similarity index over the image is: 0.9831090534102505


-----

# Latent Space

In [57]:
# Collect latent representations for healthy and outlier data
latent_representations_healthy = collect_latent_representations(model, dataset_healthy)
latent_representations_outlier = collect_latent_representations(model, dataset_outlier)
# print(latent_representations_healthy[0].shape)


# Flatten the collected latent representations
latent_representations_healthy_flat = np.concatenate([latent.flatten().reshape(1, -1) for latent in latent_representations_healthy], axis=0)
latent_representations_outlier_flat = np.concatenate([latent.flatten().reshape(1, -1) for latent in latent_representations_outlier], axis=0)

merged = np.concatenate((latent_representations_healthy_flat, latent_representations_outlier_flat), axis=0)
target = np.zeros((merged.shape[0], 1))
target[120:] = 1

target2 = []
for i in range(merged.shape[0]):
    if i < 120:
        target2.append('Healthy')
    else:
        target2.append('Outlier')
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target2)


-----

# PCA

In [58]:

pca = PCA(n_components=2) 
# pca.fit(merged)
pca.fit(latent_representations_healthy_flat)

# Transform
latent_pca = pca.transform(merged)


## Plotting the PCA visualization
color_mapping = {'Healthy': 'blue', 'Outlier': 'red'}
colors = [color_mapping[label] for label in target2]

plt.figure(figsize=(10, 6))
scatter = plt.scatter(latent_pca[:, 0], latent_pca[:, 1], c=colors, alpha=0.5) #, cmap='viridis')

plt.title('PCA Visualization of Latent Space Representations')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Healthy', markersize=10, markerfacecolor='blue'),
                   Line2D([0], [0], marker='o', color='w', label='Outlier', markersize=10, markerfacecolor='red')]
plt.legend(handles=legend_elements)

plt.show()

In [59]:
# Perform PCA with 3 components
pca = PCA(n_components=3)
pca.fit(merged)
# pca.fit(latent_representations_healthy_flat)

#Transform
latent_pca = pca.transform(merged)

## Plotting the PCA visualization
color_mapping = {'Healthy': 'blue', 'Outlier': 'red'}
colors = [color_mapping[label] for label in target2]

# Plot
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(latent_pca[:, 0], latent_pca[:, 1], latent_pca[:, 2], c=colors, alpha=0.5)

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Healthy', markersize=10, markerfacecolor='blue'),
                   Line2D([0], [0], marker='o', color='w', label='Outlier', markersize=10, markerfacecolor='red')]
plt.legend(handles=legend_elements)

ax.set_title('PCA Visualization of Latent Space Representations (3D)')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.legend(handles=legend_elements)
plt.show()

----

# TSNE

In [60]:
tsne = TSNE(n_components=2, random_state=42, perplexity=40, init='pca', learning_rate=100, n_iter=10000)

# Transform
latent_tsne = tsne.fit_transform(merged)

## Plotting the t-SNE visualization
color_mapping = {'Healthy': 'blue', 'Outlier': 'red'}
colors = [color_mapping[label] for label in target2]

plt.figure(figsize=(10, 6))
scatter = plt.scatter(latent_tsne[:, 0], latent_tsne[:, 1], c=colors, alpha=0.5) #, cmap='viridis')

plt.title('TSNE Visualization of Latent Space Representations')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Healthy', markersize=10, markerfacecolor='blue'),
                   Line2D([0], [0], marker='o', color='w', label='Outlier', markersize=10, markerfacecolor='red')]
plt.legend(handles=legend_elements)

plt.show()


In [61]:
# Perform tsne with 3 components
tsne = TSNE(n_components=3, random_state=42, perplexity=30, learning_rate=100) 

# Transform
latent_tsne = tsne.fit_transform(merged)

## Plotting the t-SNE visualization
color_mapping = {'Healthy': 'blue', 'Outlier': 'red'}
colors = [color_mapping[label] for label in target2]

# Plot
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(latent_tsne[:, 0], latent_tsne[:, 1], latent_tsne[:, 2], c=colors, alpha=0.5)

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Healthy', markersize=10, markerfacecolor='blue'),
                   Line2D([0], [0], marker='o', color='w', label='Outlier', markersize=10, markerfacecolor='red')]
plt.legend(handles=legend_elements)

ax.set_title('TSNE Visualization of Latent Space Representations (3D)')
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_zlabel('t-SNE 3')
ax.legend(handles=legend_elements)
plt.show()

### Evaluate point with t-SNE model

In [56]:
## Loading test image + outlier image

# Loading data (testset, 41 elements)
img_dir_test = "C:/Users/julie/Bachelor_data/crops_test_prep/img"
heatmap_dir_test = "C:/Users/julie/Bachelor_data/crops_test_prep/heatmaps"
msk_dir_test = "C:/Users/julie/Bachelor_data/crops_test_prep/msk"

VerSe_test = LoadData(img_dir=img_dir_test, msk_dir = msk_dir_test, distfield_dir=heatmap_dir_test)
test_loader = DataLoader(VerSe_test, batch_size=1, shuffle=False, num_workers=0)

healthy2, outlier2 = healthy_outlier(test_loader)

# Create datasets
dataset_healthy2 = generate_dataset(healthy, 1)
dataset_outlier2 = generate_dataset_outlier(outlier, 1, radius=30, mode="")

In [57]:
# Collect latent representations for healthy and outlier data
latent_representations_healthy = collect_latent_representations(model, dataset_healthy)
latent_representations_outlier = collect_latent_representations(model, dataset_outlier)


# Flatten the collected latent representations
latent_representations_healthy_flat = np.concatenate([latent.flatten().reshape(1, -1) for latent in latent_representations_healthy], axis=0)
latent_representations_outlier_flat = np.concatenate([latent.flatten().reshape(1, -1) for latent in latent_representations_outlier], axis=0)

In [ ]:
from sklearn.neighbors import NearestNeighbors


# Combine healthy and outlier data
combined_data = np.vstack((latent_representations_healthy_flat, latent_representations_outlier_flat))
combined_tsne = np.vstack((latent_healthy_tsne, latent_outlier_tsne))

# Labels to distinguish between healthy and outlier data
healthy_labels = np.zeros(latent_healthy_tsne.shape[0])
outlier_labels = np.ones(latent_outlier_tsne.shape[0])
combined_labels = np.concatenate((healthy_labels, outlier_labels))

# New point in the high-dimensional space
new_point = ...  # your new high-dimensional point

# Fit k-NN on the combined high-dimensional data
knn = NearestNeighbors(n_neighbors=5, algorithm='auto')
knn.fit(combined_data)

# Find the nearest neighbors for the new point
distances, indices = knn.kneighbors([new_point])

# Retrieve the t-SNE coordinates and labels of the nearest neighbors
neighbors_tsne_coords = combined_tsne[indices[0]]
neighbor_labels = combined_labels[indices[0]]

# Perform weighted average based on distances (inverse of distances as weights)
weights = 1 / distances[0]
normalized_weights = weights / np.sum(weights)
new_point_tsne_coords = np.dot(normalized_weights, neighbors_tsne_coords)

print("Estimated t-SNE coordinates of the new point:", new_point_tsne_coords)

# Determine the cluster of the new point using majority vote
from collections import Counter
cluster_counter = Counter(neighbor_labels)
new_point_cluster = cluster_counter.most_common(1)[0][0]

print("Assigned cluster of the new point:", "Healthy" if new_point_cluster == 0 else "Outlier")
